<a href="https://colab.research.google.com/github/ominivac/analise_integraSUS/blob/master/Analise%20Descritiva%20IntegraSUS_PLOTLY_Roberto_Sousa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook Pandas para Análise Descritiva - concurso INTEGRASUS 2020**
Fonte dos dados analisados :  http://download-integrasus.saude.ce.gov.br/download

Paticipante : Roberto Sousa - robertokbs@gmail.com

In [1]:
import folium

import psycopg2
import numpy as np
import pandas as pd
import altair as alt
from datetime import *
import plotly.express as px
import plotly.graph_objects as go



/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/IntegraSUS/casos_coronavirus_2021_01_16.csv", low_memory=False, sep=';', date_parser= '%Y-%m-%d')

In [4]:
#df.info()

In [5]:
#quantidade de registros
print('Nº de registros : ' , df.shape )

Nº de registros :  (1629146, 62)


In [6]:
# Preview dos dados , indentificação de colunas, fortmato dos dados etc
df.head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito
0,83db2a7b8011265ab0acd6522fb7862b,NaN,232004046659,NaN,NaN,Positivo,Confirmado Laboratorial,Cura,NaN,False,NaN,CE,230970.0,PACATUBA,MONGUBA,NaN,MASCULINO,20.0,20 a 24 anos,1999-11-12,2020-06-15 00:00:00,2020-05-31,NaN,2020-06-15 00:00:00,2020-06-15 00:00:00,Positivo,ESUS (TESTE RÁPIDO),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,Indígena,NaN,7216262,PACATUBA,NaN,NaN,NaN,TESTE RÁPIDO - ANTICORPO,NaN,NaN,NaN,NaN,NaN
1,5365ef99c663cf1115712adc3722d72a,NaN,232023236962,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,CE,230440.0,FORTALEZA,CONJUNTO PALMEIRAS,NaN,MASCULINO,13.0,10 a 14 anos,2007-02-14,2020-08-30 00:00:00,2020-08-22,NaN,NaN,NaN,Provável,ESUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Não Informado,NaN,Parda,NaN,7429398,FORTALEZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38b518977d1f37702dd1366b3eeff508,NaN,4120014881584,NaN,NaN,Negativo,NaN,NaN,NaN,False,NaN,CE,230440.0,FORTALEZA,CANIDEZINHO,NaN,NaN,22.0,20 a 24 anos,1998-06-15,2020-07-30 00:00:00,2020-07-10,NaN,2020-07-21 00:00:00,2020-07-21 00:00:00,Negativo,ESUS (TESTE RÁPIDO),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,NaN,NaN,9225323,CURITIBA,NaN,NaN,NaN,TESTE RÁPIDO - ANTICORPO,NaN,NaN,NaN,NaN,NaN


In [7]:
df.dtypes

identificadorCaso             object
idRedcap                     float64
idEsus                        object
idSivep                      float64
classificacaoEstadoRedcap    float64
                              ...   
obitoConfirmado               object
dataObito                     object
classificacaoObito            object
localObito                    object
tipoLocalObito                object
Length: 62, dtype: object

In [8]:
# Tranformar as col. de data no tipo certo
df[["dataNotificacao","dataSolicitacaoExame","dataResultadoExame","dataColetaExame","dataInicioSintomas", "dataObito", "dataNotificacaoObito"]] =  df[["dataNotificacao","dataSolicitacaoExame","dataResultadoExame","dataColetaExame","dataInicioSintomas", "dataObito", "dataNotificacaoObito"]].apply(pd.to_datetime,  errors='coerce')

In [9]:
df.dtypes

identificadorCaso                    object
idRedcap                            float64
idEsus                               object
idSivep                             float64
classificacaoEstadoRedcap           float64
                                  ...      
obitoConfirmado                      object
dataObito                    datetime64[ns]
classificacaoObito                   object
localObito                           object
tipoLocalObito                       object
Length: 62, dtype: object

**1.1 Distribuição de atributos**

In [10]:
# Algumas colunas em brancos, contar para sexoPaciente
count = df["sexoCaso"].isnull().sum()
print(count)

4137


In [11]:
 # Ver distribuição de atributos
df['racaCor'].unique().tolist()

['Indígena',
 'Parda',
 nan,
 'Branca',
 'Preta',
 'Amarela',
 'Ignorado',
 'Sem Informacao']

In [12]:
# Alterar onde for nan para Não Informado
df['racaCor'] = df['racaCor'].fillna('Não Informado')

In [13]:
df['tipoTesteExame'].unique().tolist()

['Teste Rápido',
 'Não Informado',
 'RT-PCR',
 nan,
 'Quimioluminescência',
 'Confirmado Clínico-Epidemiológico',
 'CLIA',
 'ECLIA',
 'Confirmado por Critério Clínico',
 'ELISA',
 'Eletroquimioluminescência',
 'Confirmado Clínico-Imagem',
 'Imunocromatografia']

In [14]:
df['evolucaoCasoSivep'].unique().tolist()

[nan, 'Cura', 'Óbito', 'Ignorado', '3']

In [15]:
df['comorbidadePneumopatiaSivep'].unique().tolist()

[nan, 'Sim', 'Ignorado', 'Não']

In [16]:
df['tipoTesteExame'].unique().tolist()

['Teste Rápido',
 'Não Informado',
 'RT-PCR',
 nan,
 'Quimioluminescência',
 'Confirmado Clínico-Epidemiológico',
 'CLIA',
 'ECLIA',
 'Confirmado por Critério Clínico',
 'ELISA',
 'Eletroquimioluminescência',
 'Confirmado Clínico-Imagem',
 'Imunocromatografia']

In [17]:
df['resultadoFinalExame'].unique().tolist()

['Positivo', 'Provável', 'Negativo', nan, 'Em Análise', 'Inconclusivo']

In [18]:
df['evolucaoCasoEsus'].unique().tolist()

['Cura',
 nan,
 'Em tratamento domiciliar',
 'Ignorado',
 'Óbito',
 'Internado',
 'Internado em UTI']

**1.2 - CRIAÇÃO DE NOVAS COLUNAS PARA FACILITAÇÃO DE CÁLCULOS POSTERIORES**

In [19]:
# Criação da coluna faixa de idade para calculo em percentual
# Não á mais necessidade, o dataset já vem com uma coluna de faixa etária

#bins= [0,4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,80]
#labels = ['0-4', '5-9', '10-14','15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69 ','70-74', '75-79', '80+']

#df['faixaIdade'] = pd.cut(df['idadeCaso'], bins=bins, labels=labels)
#df.head(3)

**O que define o óbito?** 

In [20]:
# numero oficial : 9.933

 
#df[df['obitoConfirmado'] == True]                                                                                               11658   rows  01 -
#df[df['dataObito'].notnull()]                                                                                                   14738   rows  02 - 
#df[df['evolucaoCasoSivep'] == 'Óbito']                                                                                          11332 	 rows  03 - o mais perto do oficial
#df[ (df['obitoConfirmado'] == True) & (df['dataObito'].notnull())  ]                                                            11658 #df[ (df['dataObito'].notnull() ) & (df['resultadoFinalExame'] == 'Positivo' )]                          11659 rows
#df[ (df['obitoConfirmado'] == True) & (df['dataObito'].notnull() ) & (df['resultadoFinalExame'] == 'Positivo')   ]              11658 combinar 04 e 05 da o mesmo result. 



**Recuperados : casos ja´confirmados e que não evoluiram pra óbito**

In [21]:
# df[(df['resultadoFinalExame'] == 'Positivo') & (df['dataObito'].isnull()  )  ]      # 402.102
# df[(df['evolucaoCasoEsus'] == 'Cura') ]                                            # 478.215 / 265.528 
# df[(df['resultadoFinalExame'] == 'Positivo') & (df['evolucaoCasoEsus'] == 'Cura'  )  ] # 179.657 rows / 265.528       #USAR ESSE

**Casos confirmados - PODEM OU NÃO TER EVOLUIDO PRA ÓBITOS**

In [22]:
#df[(df['resultadoFinalExame'] == 'Positivo')] # 413.761 rows / 327.865 IntegraSUS

In [23]:
#df.sort_values(by="dataObito")

**Nova coluna quantidade de óbito para cada dia, visto que em um mesmo dia há mais de um óbito**

In [24]:
# Como cada dia representa um óbito, criar nova coluna para cada registro somente HUM por registro 
df['qtObito'] = np.where(df['obitoConfirmado'] == True, 1, 0)
#df[df['qtObito'] == 1].head(3)

**Nova coluna quantidade de casos confirmados para cada dia, visto que em um mesmo dia há mais de um caso confirmado**

In [25]:
# Nova coluna para somar de casos confirmados SEM óbito
df['qtCasoConfirmado'] = np.where(  ( df['resultadoFinalExame'] == 'Positivo')  , 1, 0) 
#df[df['qtCasoConfirmado'] == 1].head(3)

**Nova coluna quantidade de casos confirmados SEM ÓBITO**

In [26]:
# Nova coluna para somar de casos confirmados SEM óbito
df['qtCasoConfirmadoSemObito'] = np.where(  ( df['resultadoFinalExame'] == 'Positivo')  & ( df['evolucaoCasoEsus'] == 'Cura' ) , 1, 0) 
#df[df['qtCasoConfirmadoSemObito'] == 1].head(3)

**1.3 - CRIAÇÃO DE NOVO DATASET PARA CASOS CONFIRMADOS QUE PODEM OU NÃO TER EVOLUÍDOS PRA ÓBITOS**

In [27]:
df_confirmados = df[ df['resultadoFinalExame'] == 'Positivo'  ]
df_confirmados.sort_values(by="dataObito").head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
22486,e00481f300496b7c996f781c5b9edc0f,NaN,202017481,3.158525e+11,NaN,Positivo,Confirmação Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,COCO,NaN,FEMININO,84.0,80 ou mais,1936-02-26,2020-03-24,2020-03-21,NaT,2020-03-24,2020-03-24,Positivo,GAL,Positivo,NaN,NaN,NaN,Óbito,2020-03-24 00:00:00.000,Não,Sim,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,NaN,NaN,RT-PCR,2.022130e+11,Não Informado,2020-05-20 17:24:35.783,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL SAO MATEUS,NaN,1,1,0
517611,82563a758fad9d2126eabf2d356277d6,NaN,202017950,3.158514e+11,NaN,Positivo,Confirmação Laboratorial,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0
453622,82563a758fad9d2126eabf2d356277d6,NaN,202017974,3.158514e+11,NaN,NaN,NaN,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,NaN,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0


In [29]:
df_confirmados.shape       # 450.151  / ? INTEGRASUS   16/jan

(450151, 65)

**1.4 - CRIAÇÃO DE NOVO DATASET PARA CASOS CONFIRMADOS COM ÓBITO**

In [30]:
df_obitos = df[ df['obitoConfirmado'] == True]
df_obitos.sort_values(by="dataObito").head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
453622,82563a758fad9d2126eabf2d356277d6,NaN,202017974,3.158514e+11,NaN,NaN,NaN,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,NaN,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0
22486,e00481f300496b7c996f781c5b9edc0f,NaN,202017481,3.158525e+11,NaN,Positivo,Confirmação Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,COCO,NaN,FEMININO,84.0,80 ou mais,1936-02-26,2020-03-24,2020-03-21,NaT,2020-03-24,2020-03-24,Positivo,GAL,Positivo,NaN,NaN,NaN,Óbito,2020-03-24 00:00:00.000,Não,Sim,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,NaN,NaN,RT-PCR,2.022130e+11,Não Informado,2020-05-20 17:24:35.783,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL SAO MATEUS,NaN,1,1,0
92020,ed2d67f326d60173a53ee585606b5910,NaN,202016773,3.158931e+11,NaN,Positivo,Confirmação Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,FATIMA,NaN,FEMININO,41.0,40 a 44 anos,1979-02-04,2020-03-23,2020-03-18,NaT,2020-03-23,2020-03-23,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,NaN,NaN,Óbito,2020-03-24 00:00:00.000,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,RT-PCR,2.001430e+11,Não Informado,2020-05-20 17:24:52.363,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL REGIONAL UNIMED,PRIVADA,1,1,0


In [31]:
df_obitos.shape   # 11.907 óbitos

(11907, 65)

In [32]:
df_grafico_obito = df_obitos.groupby('dataObito', sort=False)["qtObito"].sum().reset_index(name ='total_obito_dia')
df_grafico_obito.sort_values('dataObito')

,dataObito,total_obito_dia
3,2020-03-24,4
294,2020-03-25,2
205,2020-03-26,4
285,2020-03-27,3
296,2020-03-28,2
...,...,...
273,2021-01-10,6
272,2021-01-11,7
282,2021-01-12,3
281,2021-01-13,7


In [33]:
df_gr_obito = (pd.to_datetime(df_obitos['dataObito'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('data_obito')
       .reset_index(name='total_obito_dia'))

df_gr_obito.sort_values('data_obito')

,data_obito,total_obito_dia
288,2020-03-24,4
295,2020-03-25,2
285,2020-03-26,4
292,2020-03-27,3
293,2020-03-28,2
...,...,...
276,2021-01-10,6
271,2021-01-11,7
291,2021-01-12,3
266,2021-01-13,7


In [34]:
#df_gr_obito['MA7'] = df_gr_obito.rolling(window=7)['total_obito_dia'].mean()


In [95]:
df_gr_obito['MA7'] = df_gr_obito.iloc[:,1].rolling(window=3).mean()

In [96]:
df_gr_obito

,data_obito,total_obito_dia,MA7,MA30,total_obito_acumulado
288,2020-03-24,4,NaN,6.300000,4
295,2020-03-25,2,NaN,5.166667,6
285,2020-03-26,4,3.333333,6.700000,10
292,2020-03-27,3,3.000000,5.700000,13
293,2020-03-28,2,3.000000,5.500000,15
...,...,...,...,...,...
276,2021-01-10,6,9.333333,7.800000,11888
271,2021-01-11,7,7.333333,8.400000,11895
291,2021-01-12,3,5.333333,5.866667,11898
266,2021-01-13,7,5.666667,8.900000,11905


In [104]:
df_gr_obito.sort_values('data_obito', inplace= True)
df_gr_obito

,data_obito,total_obito_dia,MA7,MA30,total_obito_acumulado
288,2020-03-24,4,NaN,NaN,4
295,2020-03-25,2,NaN,NaN,6
285,2020-03-26,4,3.333333,NaN,10
292,2020-03-27,3,3.000000,NaN,13
293,2020-03-28,2,3.000000,3.111111,15
...,...,...,...,...,...
276,2021-01-10,6,9.333333,8.777778,11888
271,2021-01-11,7,7.333333,8.555556,11895
291,2021-01-12,3,5.333333,7.333333,11898
266,2021-01-13,7,5.666667,6.111111,11905


In [105]:
df_gr_obito['MA30'] = df_gr_obito.iloc[:,2].rolling(window=30).mean()

In [106]:
df_gr_obito.sort_values('data_obito', inplace= True)
df_gr_obito

,data_obito,total_obito_dia,MA7,MA30,total_obito_acumulado
288,2020-03-24,4,NaN,NaN,4
295,2020-03-25,2,NaN,NaN,6
285,2020-03-26,4,3.333333,NaN,10
292,2020-03-27,3,3.000000,NaN,13
293,2020-03-28,2,3.000000,NaN,15
...,...,...,...,...,...
276,2021-01-10,6,9.333333,10.977778,11888
271,2021-01-11,7,7.333333,10.722222,11895
291,2021-01-12,3,5.333333,10.344444,11898
266,2021-01-13,7,5.666667,10.011111,11905


In [107]:
from plotly.subplots import make_subplots
# Plot 
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(df_gr_obito, x='data_obito', y='total_obito_dia',)

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA'])

# Show plot 

fig.add_trace(go.Scatter(x= df_gr_obito['data_obito'], y= df_gr_obito['MA7'], name='Média Móvel de óbitos de uma semana',
                        line=dict(color='red',width=2)))

fig.add_trace(go.Scatter(x= df_gr_obito['data_obito'], y= df_gr_obito['MA30'], name='Média Móvel de um mês',
                        line=dict(color='orange',width=2)))

fig.update_layout(title='Quantidade de óbitos confirmados no período',
                   xaxis_title='Mês',
                   yaxis_title='Quantidade de óbitos / dia')
fig.show()

**1.5 Gráfico de óbitos por dia**

In [101]:
fig = go.Figure(data=[
    go.Bar(x= df_grafico_obito['dataObito'], y= df_grafico_obito['total_obito_dia'], marker=dict(color='orangered')),
])

# gráfico
fig.update_layout(
    title='Total de óbitos de Covid-19 por dia no estado do Ceará',
    xaxis_tickfont_size=24,
    xaxis = dict(
        title='Período',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis = dict(
        title='Número de óbitos',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis_tickangle=90)

fig.show()

**Metodo para agrupar casos confirmados por dia**

In [108]:
# https://stackoverflow.com/questions/48961892/python-pandas-group-by-day-and-count-for-each-day

df_gr_conf = (pd.to_datetime(df_confirmados['dataResultadoExame'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('data_result_exame')
       .reset_index(name='total_caso_conf_dia'))
#print (df_gr_conf)
df_gr_conf.sort_values('data_result_exame')

,data_result_exame,total_caso_conf_dia
325,2020-01-01,5
315,2020-01-02,8
323,2020-01-03,5
308,2020-01-04,19
306,2020-01-05,20
...,...,...
97,2021-01-11,1704
160,2021-01-12,1249
162,2021-01-13,1219
161,2021-01-14,1240


In [112]:
df_gr_conf['MA7'] = df_gr_conf.iloc[:,1].rolling(window=7).mean()


In [113]:
df_gr_conf.sort_values('data_result_exame', inplace= True)
df_gr_conf

,data_result_exame,total_caso_conf_dia,MA7
325,2020-01-01,5,NaN
315,2020-01-02,8,NaN
323,2020-01-03,5,NaN
308,2020-01-04,19,NaN
306,2020-01-05,20,NaN
...,...,...,...
97,2021-01-11,1704,1192.428571
160,2021-01-12,1249,1176.714286
162,2021-01-13,1219,1172.000000
161,2021-01-14,1240,1101.857143


In [114]:
df_gr_conf['MA30'] = df_gr_conf.iloc[:,2].rolling(window=30).mean()

In [115]:
df_gr_conf.sort_values('data_result_exame', inplace= True)
df_gr_conf

,data_result_exame,total_caso_conf_dia,MA7,MA30
325,2020-01-01,5,NaN,NaN
315,2020-01-02,8,NaN,NaN
323,2020-01-03,5,NaN,NaN
308,2020-01-04,19,NaN,NaN
306,2020-01-05,20,NaN,NaN
...,...,...,...,...
97,2021-01-11,1704,1192.428571,1174.342857
160,2021-01-12,1249,1176.714286,1168.352381
162,2021-01-13,1219,1172.000000,1160.576190
161,2021-01-14,1240,1101.857143,1153.314286


In [116]:
from plotly.subplots import make_subplots
# Plot 
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(df_gr_conf, x='data_result_exame', y='total_caso_conf_dia',)

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA'])

# Show plot 

fig.add_trace(go.Scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA7'], name='Média Móvel de uma semana', mode='lines',
                        line=dict(color='red',width=2)))

fig.add_trace(go.Scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA30'], name='Média Móvel de um mês', mode='lines',
                        line=dict(color='orange',width=2)))

fig.update_layout(title='Quantidade de casos confirmados no período',
                   xaxis_title='Mês',
                   yaxis_title='Quantidade de casos / dia')
fig.show()

**1.4 Gráfico casos confirmados por dia**

**Metodo para agrupar por dia a quantidade de casos confirmados**

In [ ]:
df_gr_obito.head(2)

In [51]:
df_gr_obito['total_obito_acumulado'] = df_gr_obito['total_obito_dia'].cumsum()
df_gr_obito

,data_obito,total_obito_dia,MA7,MA30,total_obito_acumulado
288,2020-03-24,4,4.428571,6.300000,4
295,2020-03-25,2,2.857143,5.166667,6
285,2020-03-26,4,5.285714,6.700000,10
292,2020-03-27,3,3.714286,5.700000,13
293,2020-03-28,2,3.428571,5.500000,15
...,...,...,...,...,...
276,2021-01-10,6,6.571429,7.800000,11888
271,2021-01-11,7,7.000000,8.400000,11895
291,2021-01-12,3,3.857143,5.866667,11898
266,2021-01-13,7,7.571429,8.900000,11905


**GERAR GRÁFICO DE ÓBITOS E CASOS CONFIRMADOS POR DIA**

**Gerar grupo de faixa de idade por óbito**

In [52]:
df_obitos.head(2)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
5478,93dcfbd872b61a07b6b50f072570195c,NaN,232000773339,3.158948e+11,NaN,NaN,NaN,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,GRANJA PORTUGAL,NaN,MASCULINO,68.0,65 a 69 anos,1952-09-04,2020-05-06,2020-05-03,NaT,2020-05-10 03:00:00,2020-05-06,Positivo,LACEN,Positivo,2020-05-10 00:00:00.000,NaN,NaN,Óbito,2020-05-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,NaN,Parda,2020-05-22 15:36:36.170,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,NaN,True,2020-05-20,Confirmado,HOSPITAL ANTONIO PRUDENTE,PRIVADA,1,1,0
8289,65390b27c1653d5697e017004cf72e2e,NaN,232000359812,3.158758e+11,NaN,Positivo,Confirmado Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JACARECANGA,NaN,FEMININO,80.0,80 ou mais,1940-03-17,2020-04-22,2020-04-20,NaT,2020-04-21 00:00:00,2020-04-21,Positivo,GAL,Positivo,2020-04-21 00:00:00.000,2020-04-23 00:00:00.000,NaN,Óbito,2020-04-26 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.001610e+11,Não Informado,2020-05-20 17:28:55.050,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-04-26,Confirmado,UNIDADE PRONTO ATENDIMENTO CRISTO REDENTOR,PÚBLICA,1,1,0


In [53]:
counts_obitos_faixa_idade = df_obitos.groupby(['faixaEtaria', 'sexoCaso']).faixaEtaria.count().unstack()
counts_obitos_faixa_idade

sexoCaso,FEMININO,MASCULINO
faixaEtaria,,
00 a 04 anos,23,14
05 a 09 anos,2,5
10 a 14 anos,3,6
15 a 19 anos,14,20
20 a 24 anos,32,21
25 a 29 anos,29,54
30 a 34 anos,53,74
35 a 39 anos,84,142
40 a 44 anos,96,209


In [54]:
#type(counts_obitos_faixa_idade)

In [55]:
df1 = counts_obitos_faixa_idade.reset_index()
df1


sexoCaso,faixaEtaria,FEMININO,MASCULINO
0,00 a 04 anos,23,14
1,05 a 09 anos,2,5
2,10 a 14 anos,3,6
3,15 a 19 anos,14,20
4,20 a 24 anos,32,21
5,25 a 29 anos,29,54
6,30 a 34 anos,53,74
7,35 a 39 anos,84,142
8,40 a 44 anos,96,209
9,45 a 49 anos,142,247


In [56]:
import plotly.express as px

sexos = ['MASCULINO', 'FEMININO']
faixas  = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49',
           '50-54', '55-59', '60-64', '65-69 ', '70-74', '75-79', '80+']



fig = go.Figure(data=[
    go.Bar(name='MASCULINO', x= faixas, y= df1['MASCULINO']),
    go.Bar(name='FEMININO', x= faixas, y= df1['FEMININO'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='Total de óbitos por faixa de idade')
fig.show()





Distribuição de óbitos por sexo do paciente

In [57]:
obito_by_sexo = df_obitos.groupby([ 'sexoCaso', 'racaCor']).size().unstack('racaCor')
obito_by_sexo.columns =  ['Parda','Branca','Amarela', 'Não Informado','Preta','Ignorado','Indígena', 'Indigena']

In [58]:
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indígena,Indigena
sexoCaso,,,,,,,,
FEMININO,48,395,10,2,2074,2390,56,21
MASCULINO,71,452,12,12,2769,3463,101,31


In [59]:
obito_by_sexo['ind'] = obito_by_sexo['Indígena']+obito_by_sexo['Indigena']
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indígena,Indigena,ind
sexoCaso,,,,,,,,,
FEMININO,48,395,10,2,2074,2390,56,21,77
MASCULINO,71,452,12,12,2769,3463,101,31,132


In [60]:
obito_by_sexo.drop(['Indígena', 'Indigena'], axis=1, inplace= True)
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,ind
sexoCaso,,,,,,,
FEMININO,48,395,10,2,2074,2390,77
MASCULINO,71,452,12,12,2769,3463,132


In [61]:
obito_by_sexo.rename(columns={"ind": "Indigena"}, inplace= True)
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indigena
sexoCaso,,,,,,,
FEMININO,48,395,10,2,2074,2390,77
MASCULINO,71,452,12,12,2769,3463,132


In [62]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ['FEMININO', 'MASCULINO']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=2, cols=3, specs=[ [ {'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
                                          [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]
                                          ]
                    )

#parda
fig.add_trace(go.Pie(labels=labels, values=[45,68], name="Parda"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Branca"), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=[9,6], name="Amarela"), 1, 3)

fig.add_trace(go.Pie(labels=labels, values=[2,12], name="Não Informado"), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=[9,6], name="Preta"), 2, 2)
fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Ignorado"),2, 3)

#fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Ignorado"),3, 1)
#branca

#fig.add_trace(go.Pie(labels=labels, values=[68, 381], name="Parda"), 2, 1)
#fig.add_trace(go.Pie(labels=labels, values=[68, 441], name="Branca"),2, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4) #hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Relação de Número de óbitos confirmados segundo raça/cor",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Parda', x=0.12, y=0.5, font_size=20, showarrow=False),
                 dict(text='Branca', x=0.50, y=0.5, font_size=20, showarrow=False),
                 dict(text='Amarela', x=0.90, y=0.5, font_size=20, showarrow=False),
                  dict(text='Nao Informado', x=0.12, y=3.4, font_size=20, showarrow=False)]
                  )
fig.show()



In [63]:
counts_casos_raca_cor = df_obitos.groupby(['racaCor', 'sexoCaso']).racaCor.count().unstack()
counts_casos_raca_cor

sexoCaso,FEMININO,MASCULINO
racaCor,,
Amarela,48,71
Branca,395,452
Ignorado,10,12
Indígena,2,12
Não Informado,2074,2769
Parda,2390,3463
Preta,56,101
Sem Informacao,21,31


In [64]:
df_counts_casos_raca_cor = counts_casos_raca_cor.reset_index()
df_counts_casos_raca_cor

sexoCaso,racaCor,FEMININO,MASCULINO
0,Amarela,48,71
1,Branca,395,452
2,Ignorado,10,12
3,Indígena,2,12
4,Não Informado,2074,2769
5,Parda,2390,3463
6,Preta,56,101
7,Sem Informacao,21,31


In [65]:
import plotly.express as px

sexos = ['MASCULINO', 'FEMININO']
faixas  = ['Amarela	', 'Branca', 'Ignorado', 'Indígena', 'Não Informado', 'Parda', 'Preta', 'Sem Informação']



fig = go.Figure(data=[
    go.Bar(name='MASCULINO', x= df_counts_casos_raca_cor.racaCor	, y= df_counts_casos_raca_cor['MASCULINO']),
    go.Bar(name='FEMININO', x= df_counts_casos_raca_cor.racaCor, y= df_counts_casos_raca_cor['FEMININO'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='Total de óbitos por faixa de idade')
fig.show()

In [66]:
df_confirmados_por_municipio = pd.read_csv("/content/drive/My Drive/IntegraSUS/casos_coronavirus_localidade_2021_01_10.csv", low_memory=False, sep=';')

In [67]:
df_confirmados_por_municipio.head(2)

,paisCaso,estadoCaso,municipioCaso,codigoMunicipioCaso,bairroCaso,cepCaso
0,NaN,CE,FORTALEZA,230440.0,BARRA DO CEARA,60332690.0
1,NaN,CE,FORTALEZA,230440.0,NaN,NaN


In [68]:
# Como cada linha representa um caso, criar nova coluna com quantidade um para posterior agrupamento 
df_confirmados_por_municipio['qt_caso'] = np.where(df['municipioCaso'].notnull, 1, 0)


In [69]:
df_confirmados_por_municipio.head(3)

,paisCaso,estadoCaso,municipioCaso,codigoMunicipioCaso,bairroCaso,cepCaso,qt_caso
0,NaN,CE,FORTALEZA,230440.0,BARRA DO CEARA,60332690.0,1
1,NaN,CE,FORTALEZA,230440.0,NaN,NaN,1
2,NaN,CE,IGUATU,230550.0,PENHA,NaN,1


In [70]:
df_grafico_confirmados_municipio = df_confirmados_por_municipio.groupby('municipioCaso', sort=False)["qt_caso"].sum().reset_index(name ='total_caso_municipio')

In [71]:
df_grafico_confirmados_municipio.sort_values('total_caso_municipio')
df_grafico_confirmados_municipio

,municipioCaso,total_caso_municipio
0,FORTALEZA,541919
1,IGUATU,27033
2,MARANGUAPE,17130
3,ICO,14545
4,MORADA NOVA,9685
...,...,...
2057,MATARACA,1
2058,SAO MANUEL,1
2059,JAGUARI,1
2060,GOVERNADOR ARCHER,1


In [72]:
type(df_grafico_confirmados_municipio)

pandas.core.frame.DataFrame

In [73]:
df_grafico_confirmados_municipio[df_grafico_confirmados_municipio['municipioCaso'] == 'FORTALEZA']

,municipioCaso,total_caso_municipio
0,FORTALEZA,541919


In [74]:


fig = go.Figure(data=[
    go.Bar(x= df_grafico_confirmados_municipio['municipioCaso'], y=  df_grafico_confirmados_municipio['total_caso_municipio'], marker=dict(color='orangered')),
])

# gráfico
fig.update_layout(
    title='Total de casos confiramdos distribuiídos por raça do paciente no estado do Ceará',
    xaxis_tickfont_size=24,
    xaxis = dict(
        title='Município',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis = dict(
        title='Número de casos confirmados',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis_tickangle=90)

fig.show()

In [75]:
import plotly.express as px


fig = px.bar(df_grafico_confirmados_municipio, x= df_grafico_confirmados_municipio['municipioCaso'], y= df_grafico_confirmados_municipio['total_caso_municipio'])
fig.show()